# SchNet S2EF training example

The purpose of this notebook is to demonstrate some of the basics of the Open Catalyst Project's (OCP) codebase and data. In this example, we will train a schnet model for predicting the energy and forces of a given structure (S2EF task). First, ensure you have installed the OCP ocp repo and all the dependencies according to the [README](https://github.com/Open-Catalyst-Project/ocp/blob/master/README.md).

Disclaimer: This notebook is for tutorial purposes, it is unlikely it will be practical to train baseline models on our larger datasets using this format. As a next step, we recommend trying the command line examples. 

## Imports

In [1]:
import datetime
import os
import sys
import yaml

import torch

In [2]:
from ocpmodels.common import distutils
from ocpmodels.common.flags import flags
from ocpmodels.common.registry import registry
from ocpmodels.common.utils import (
    build_config,
    create_grid,
    save_experiment_log,
    setup_imports,
)
from ocpmodels.trainers import ForcesTrainer

In [3]:
# a simple sanity check that a GPU is available
if torch.cuda.is_available():
    print("True")
else:
    print("False")

True


## The essential steps for training an OCP model

1) Download data

2) Preprocess data (if necessary)

3) Define or load a configuration (config), which includes the following
   
   - task
   - model
   - optimizer
   - dataset
   - trainer

4) Train

5) Depending on the model/task there might be intermediate relaxation step

6) Predict

## Download and preprocess data

Please download the train S2EF 200K dataset from the [DATASET.md](https://github.com/Open-Catalyst-Project/ocp/blob/master/DATASET.md) and follow directions to untar, uncompress, and preprocess in the [README.md](https://github.com/Open-Catalyst-Project/ocp/blob/master/README.md#preprocess-datasets---s2ef-only). This example assumes the --get-edges flag to be present at preprocessing time.

In [4]:
# set the path to your local lmdb directory
train_src = "/home/mshuaibi/projects/ocp/data/train"

## Define config

For this example, we will explicitly define the config; however, a set of default config files exists in the config folder of this repository. Default config yaml files can easily be loaded with the `build_config` util (found in `ocp/ocpmodels/common/utils.py`). Loading a yaml config is preferrable when launching jobs from the command line. We have included our best models' config files [here](https://github.com/Open-Catalyst-Project/ocp/tree/master/configs/s2ef).

**Task** 

In [5]:
task = {
    'dataset': 'trajectory_lmdb', # dataset used for the S2EF task
    'description': 'Regressing to energies and forces for DFT trajectories from OCP',
    'type': 'regression',
    'metric': 'mae',
    'labels': ['potential energy'],
    'grad_input': 'atomic forces',
    'train_on_free_atoms': True,
    'eval_on_free_atoms': True
}

**Model** - SchNet for this example

In [6]:
model = {
    'name': 'schnet',
    'hidden_channels': 1024, # if training is too slow for example purposes reduce the number of hidden channels
    'num_filters': 256,
    'num_interactions': 3,
    'num_gaussians': 200,
    'cutoff': 6.0
}

**Optimizer**

In [7]:
optimizer = {
    'batch_size': 16, # if hitting GPU memory issues, lower this
    'eval_batch_size': 8,
    'num_workers': 64,
    'lr_initial': 0.0001,
    'lr_gamma': 0.1,
    'lr_milestones': [15, 20],
    'warmup_epochs': 10,
    'warmup_factor': 0.2,
    'max_epochs': 1, # used for demonstration purposes
    'force_coefficient': 100,
}

**Dataset**

For simplicity, `train_src` is used for all the train/val/test sets. Feel free to update with the actual S2EF val and test sets, but it does require additional downloads and preprocessing. If you desire to normalize your targets, `normalize_labels` must be set to `True` and corresponding `mean` and `std`s need to be specified. These values have been precomputed for you and can be found in any of the [`base.yml`](https://github.com/Open-Catalyst-Project/ocp/blob/master/configs/s2ef/20M/base.yml#L5-L9) config files.

In [8]:
dataset = [
{'src': train_src, 'normalize_labels': False}, # train set 
{'src': train_src}, # val set
{'src': train_src} # test set
]

**Trainer**

Use the `ForcesTrainer` for the S2EF and IS2RS tasks, and the `EnergyTrainer` for the IS2RE task 

In [9]:
trainer = ForcesTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="SchNet-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    is_vis=False,
    print_every=10,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

amp: false
cmd:
  checkpoint_dir: ./checkpoints/2020-10-14-12-47-30-SchNet-example
  identifier: SchNet-example
  logs_dir: ./logs/tensorboard/2020-10-14-12-47-30-SchNet-example
  print_every: 10
  results_dir: ./results/2020-10-14-12-47-30-SchNet-example
  seed: 0
  timestamp: 2020-10-14-12-47-30-SchNet-example
dataset:
  normalize_labels: false
  src: /home/mshuaibi/projects/ocp/data/train
logger: tensorboard
model: schnet
model_attributes:
  cutoff: 6.0
  hidden_channels: 1024
  num_filters: 256
  num_gaussians: 200
  num_interactions: 3
optim:
  batch_size: 16
  eval_batch_size: 8
  force_coefficient: 100
  lr_gamma: 0.1
  lr_initial: 0.0001
  lr_milestones:
  - 15
  - 20
  max_epochs: 1
  num_workers: 64
  warmup_epochs: 10
  warmup_factor: 0.2
task:
  dataset: trajectory_lmdb
  description: Regressing to energies and forces for DFT trajectories from OCP
  eval_on_free_atoms: true
  grad_input: atomic forces
  labels:
  - potential energy
  metric: mae
  train_on_free_atoms: true


## Check the model

In [10]:
print(trainer.model)

OCPDataParallel(
  (module): SchNet(hidden_channels=1024, num_filters=256, num_interactions=3, num_gaussians=200, cutoff=6.0)
)


## Train

In [11]:
trainer.train()

forcesx_mae: 0.7030, forcesy_mae: 0.7115, forcesz_mae: 0.7711, forces_mae: 0.7285, forces_cos: 0.0054, forces_magnitude: 1.3615, energy_mae: 44.7269, energy_force_within_threshold: 0.0000, loss: 117.5761, epoch: 0.0016
forcesx_mae: 0.5769, forcesy_mae: 0.6908, forcesz_mae: 0.6871, forces_mae: 0.6516, forces_cos: -0.0030, forces_magnitude: 1.1874, energy_mae: 33.8378, energy_force_within_threshold: 0.0000, loss: 98.4086, epoch: 0.0176
forcesx_mae: 0.4555, forcesy_mae: 0.5435, forcesz_mae: 0.5442, forces_mae: 0.5144, forces_cos: 0.0021, forces_magnitude: 0.9170, energy_mae: 27.3140, energy_force_within_threshold: 0.0000, loss: 78.5339, epoch: 0.0336
forcesx_mae: 0.3672, forcesy_mae: 0.4374, forcesz_mae: 0.4337, forces_mae: 0.4128, forces_cos: 0.0066, forces_magnitude: 0.7168, energy_mae: 17.3954, energy_force_within_threshold: 0.0000, loss: 58.3405, epoch: 0.0496
forcesx_mae: 0.3477, forcesy_mae: 0.3784, forcesz_mae: 0.3972, forces_mae: 0.3744, forces_cos: 0.0070, forces_magnitude: 0.639

forcesx_mae: 0.1760, forcesy_mae: 0.2116, forcesz_mae: 0.2183, forces_mae: 0.2020, forces_cos: 0.0216, forces_magnitude: 0.3082, energy_mae: 3.8783, energy_force_within_threshold: 0.0000, loss: 24.0622, epoch: 0.6096
forcesx_mae: 0.1859, forcesy_mae: 0.2258, forcesz_mae: 0.2249, forces_mae: 0.2122, forces_cos: 0.0511, forces_magnitude: 0.3259, energy_mae: 4.2273, energy_force_within_threshold: 0.0000, loss: 25.4100, epoch: 0.6256
forcesx_mae: 0.1628, forcesy_mae: 0.1864, forcesz_mae: 0.1954, forces_mae: 0.1815, forces_cos: 0.0226, forces_magnitude: 0.2751, energy_mae: 3.9199, energy_force_within_threshold: 0.0000, loss: 22.1479, epoch: 0.6416
forcesx_mae: 0.1615, forcesy_mae: 0.1999, forcesz_mae: 0.2143, forces_mae: 0.1919, forces_cos: 0.0243, forces_magnitude: 0.2961, energy_mae: 3.3222, energy_force_within_threshold: 0.0000, loss: 22.5459, epoch: 0.6576
forcesx_mae: 0.1554, forcesy_mae: 0.2105, forcesz_mae: 0.2117, forces_mae: 0.1925, forces_cos: 0.0087, forces_magnitude: 0.3010, ene

100%|██████████| 1250/1250 [00:23<00:00, 54.30it/s]


forcesx_mae: 0.1379, forcesy_mae: 0.1632, forcesz_mae: 0.1689, forces_mae: 0.1567, forces_cos: 0.0433, forces_magnitude: 0.2295, energy_mae: 3.6160, energy_force_within_threshold: 0.0000, loss: 19.2852, epoch: 1.0000
### Evaluating on test.


100%|██████████| 1250/1250 [00:22<00:00, 54.54it/s]

forcesx_mae: 0.1379, forcesy_mae: 0.1632, forcesz_mae: 0.1689, forces_mae: 0.1567, forces_cos: 0.0433, forces_magnitude: 0.2295, energy_mae: 3.6160, energy_force_within_threshold: 0.0000, loss: 19.2852, epoch: 1.0000


## Predict

To make predictions on unseen data a `torch.utils.data` DataLoader object much be constructed. Here we reference our test set to make predictions on.

In [14]:
# make predictions on the existing test_loader
predictions = trainer.predict(trainer.test_loader, disable_tqdm=False)

100%|██████████| 1250/1250 [00:19<00:00, 62.52it/s]


In [18]:
energy_pred = predictions["energy"] # list of scalars
forces_pred = predictions["forces"] # list of arrays